## Code Setup

In [ ]:
# If True, assumes everything is running locally.
IS_LOCAL = False

# Path to main directory
REMOTE_PATH = "/content/gdrive/MyDrive/CA-Final"
LOCAL_PATH = "/Users/luca/Library/CloudStorage/GoogleDrive-luca@ucsc.edu/Shared drives/Birds and CS/Data/CA-Final"
DATA_PATH = LOCAL_PATH if IS_LOCAL else REMOTE_PATH

In [ ]:
import os
import sys

In [ ]:
# Installs required packages
if not IS_LOCAL:
    !pip install git+https://github.com/ecoscape-earth/ecoscape-connectivity.git
    !pip install git+https://github.com/ecoscape-earth/ecoscape-utils.git

In [ ]:
# Connecting to Drive.
if not IS_LOCAL:
    from google.colab import drive
    drive.mount("/content/gdrive", force_remount=True)

In [ ]:
import time
from ecoscape_utilities import BirdRun
from ecoscape_connectivity import compute_connectivity
from ecoscape_connectivity.util import read_transmission_csv

## Bird Run Definition

In [ ]:
def run_birds(target=None):

    bird_run = BirdRun(DATA_PATH)

    birds = []

    NUM_SIMULATIONS = 400
    HOPS = [1, 2, 3, 4, 5, 6]
    NUM_SPREADS = [2, 3, 4, 6, 8, 10, 15, 20, 30, 40]

    for hop_distance in HOPS:
        for num_spreads in NUM_SPREADS:
            birds.append(bird_run.get_bird_run(
                "acowoo", "Acorn Woodpecker",
                do_validation=IS_LOCAL, run_name=target,
                hop_distance=hop_distance, num_spreads=num_spreads,
                num_simulations=2 if IS_LOCAL else NUM_SIMULATIONS))

    for hop_distance in HOPS:
        for num_spreads in NUM_SPREADS:
            birds.append(bird_run.get_bird_run(
                "stejay", "Steller's Jay",
                do_validation=IS_LOCAL, run_name=target,
                hop_distance=hop_distance, num_spreads=num_spreads,
                num_simulations=2 if IS_LOCAL else NUM_SIMULATIONS))

    for bird in birds:

        # Creates output folder, if missing.
        bird_run.createdir_for_file(bird.repopulation_fn)
        bird_run.createdir_for_file(bird.gradient_fn)

        transmission_d = read_transmission_csv(bird.transmission_fn)

        compute_connectivity(
            habitat_fn=bird.habitat_fn,
            terrain_fn=bird.terrain_fn,
            connectivity_fn=bird.repopulation_fn,
            flow_fn=bird.gradient_fn if bird.do_gradient else None,
            permeability_dict=transmission_d,
            gap_crossing=bird.hop_distance,
            num_gaps=bird.num_spreads,
            num_simulations=bird.num_simulations,
            seed_density=4,
            single_tile=True
        )

        print("Processed", bird.name, "gradient", bird.do_gradient, "hop", bird.hop_distance, "num", bird.num_spreads, "sim", bird.num_simulations)

In [ ]:
def run_birds_detail(target=None):

    bird_run = BirdRun(DATA_PATH)

    birds = []

    NUM_SIMULATIONS = 10000

    # Uncomment below the specific simulations you wish to run.

    birds.append(bird_run.get_bird_run(
        "acowoo", "Acorn Woodpecker",
        do_validation=IS_LOCAL, run_name=target,
        hop_distance=2, num_spreads=20,
        num_simulations=2 if IS_LOCAL else NUM_SIMULATIONS))

    birds.append(bird_run.get_bird_run(
        "stejay", "Steller's Jay",
        do_validation=IS_LOCAL, run_name=target,
        hop_distance=2, num_spreads=3,
        num_simulations=2 if IS_LOCAL else NUM_SIMULATIONS))

    birds.append(bird_run.get_bird_run(
        "stejay", "Steller's Jay",
        do_validation=IS_LOCAL, run_name=target,
        hop_distance=1, num_spreads=6,
        num_simulations=2 if IS_LOCAL else NUM_SIMULATIONS))

    for bird in birds:

        # Creates output folder, if missing.
        bird_run.createdir_for_file(bird.repopulation_fn)
        bird_run.createdir_for_file(bird.gradient_fn)

        transmission_d = read_transmission_csv(bird.transmission_fn)

        compute_connectivity(
            habitat_fn=bird.habitat_fn,
            terrain_fn=bird.terrain_fn,
            connectivity_fn=bird.repopulation_fn,
            flow_fn=bird.gradient_fn if bird.do_gradient else None,
            permeability_dict=transmission_d,
            gap_crossing=bird.hop_distance,
            num_gaps=bird.num_spreads,
            num_simulations=bird.num_simulations,
            seed_density=4,
            single_tile=True
        )

        print("Processed", bird.name, "gradient", bird.do_gradient, "hop", bird.hop_distance, "num", bird.num_spreads, "sim", bird.num_simulations)

In [ ]:
def run_birds_gradient(target=None):

    bird_run = BirdRun(DATA_PATH)

    birds = []

    NUM_SIMULATIONS = 1000

    # Uncomment below the specific simulations you wish to run.

    birds.append(bird_run.get_bird_run(
        "acowoo", "Acorn Woodpecker",
        do_validation=IS_LOCAL, run_name=target,
        hop_distance=2, num_spreads=20,
        do_gradient=True,
        num_simulations=2 if IS_LOCAL else NUM_SIMULATIONS))


    birds.append(bird_run.get_bird_run(
        "stejay", "Steller's Jay",
        do_validation=IS_LOCAL, run_name=target,
        hop_distance=2, num_spreads=3,
        do_gradient=True,
        num_simulations=2 if IS_LOCAL else NUM_SIMULATIONS))

    birds.append(bird_run.get_bird_run(
        "stejay", "Steller's Jay",
        do_validation=IS_LOCAL, run_name=target,
        hop_distance=1, num_spreads=6,
        do_gradient=True,
        num_simulations=2 if IS_LOCAL else NUM_SIMULATIONS))

    for bird in birds:

        # Creates output folder, if missing.
        bird_run.createdir_for_file(bird.repopulation_fn)
        bird_run.createdir_for_file(bird.gradient_fn)

        transmission_d = read_transmission_csv(bird.transmission_fn)

        compute_connectivity(
            habitat_fn=bird.habitat_fn,
            terrain_fn=bird.terrain_fn,
            connectivity_fn=bird.repopulation_fn,
            flow_fn=bird.gradient_fn if bird.do_gradient else None,
            permeability_dict=transmission_d,
            gap_crossing=bird.hop_distance,
            num_gaps=bird.num_spreads,
            num_simulations=bird.num_simulations,
            seed_density=4,
            single_tile=False,
            tile_size=1024, tile_border=256
        )

        print("Processed", bird.name, "gradient", bird.do_gradient, "hop", bird.hop_distance, "num", bird.num_spreads, "sim", bird.num_simulations)

In [ ]:
run_birds(target="Paper")

In [ ]:
run_birds_detail(target="Paper10000")


In [ ]:
run_birds_gradient(target="Gradient")